In [1]:
import pandas as pd
import os
import shutil
from rapidfuzz import fuzz
from datetime import datetime, timezone

In [3]:
def check_for_duplicate_players_by_name(data_directory, seasons):
    duplicate_players = {}

    for season in seasons:
        season_path = os.path.join(data_directory, season)
        player_idlist_path = os.path.join(season_path, "player_idlist.csv")
        
        if not os.path.exists(player_idlist_path):
            print(f"player_idlist.csv not found for season: {season}")
            continue
        
        try:
            player_idlist = pd.read_csv(player_idlist_path)
            duplicates = player_idlist.duplicated(subset=['first_name', 'second_name'], keep=False)
            duplicate_names = player_idlist.loc[duplicates, ['first_name', 'second_name']].drop_duplicates()

            if not duplicate_names.empty:
                duplicate_players[season] = duplicate_names
                print(f"Found duplicate names in season {season}:")
                print(duplicate_names)
            else:
                print(f"No duplicate names found in season {season}.")
        
        except Exception as e:
            print(f"Error checking duplicates for season {season}: {e}")

    return duplicate_players

data_directory = "Fantasy-Premier-League/data"
seasons = ["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"]
duplicates_by_name = check_for_duplicate_players_by_name(data_directory, seasons)

No duplicate names found in season 2020-21.
No duplicate names found in season 2021-22.
No duplicate names found in season 2022-23.
No duplicate names found in season 2023-24.
No duplicate names found in season 2024-25.


In [3]:
# We found the duplicates to Ben Davies in seasons 2020-21, 2021-22 and 2022-23 and Alvaro Fernandez in 2021-22.
# We will keep the Tottenham Hotspurs Ben Davies as he is also in the current season but we can safely remove
# both Alvaro Fernandezes and the Liverpool Ben Davies since they had almost no minutes during their Premier 
# League time
def remove_players_and_folders(data_directory, players_to_remove):
    for player in players_to_remove:
            season = player["season"]
            player_id = player["id"]
            first_name = player["first_name"]
            second_name = player["second_name"]

            season_path = os.path.join(data_directory, season)
            player_idlist_path = os.path.join(season_path, "player_idlist.csv")
            players_folder_path = os.path.join(season_path, "players")

            if not os.path.exists(player_idlist_path):
                print(f"player_idlist.csv not found for season {season}. Skipping.")
                continue

            try:
                # Remove from player_idlist.csv
                player_idlist = pd.read_csv(player_idlist_path)

                # Find matching player row
                matching_rows = player_idlist[
                    (player_idlist["id"] == player_id) &
                    (player_idlist["first_name"] == first_name) &
                    (player_idlist["second_name"] == second_name)
                ]

                if not matching_rows.empty:
                    # Remove the player from the DataFrame
                    player_idlist = player_idlist.drop(matching_rows.index)
                    # Save back to CSV
                    player_idlist.to_csv(player_idlist_path, index=False)
                    print(f"Removed {first_name} {second_name} (ID: {player_id}) from {player_idlist_path}.")
                else:
                    print(f"Player {first_name} {second_name} (ID: {player_id}) not found in {player_idlist_path}. Skipping.")

                # Remove player folder
                player_folder_name = f"{first_name}_{second_name}_{player_id}"
                player_folder_path = os.path.join(players_folder_path, player_folder_name)
                if os.path.exists(player_folder_path):
                    shutil.rmtree(player_folder_path)
                    print(f"Removed folder: {player_folder_path}")
                else:
                    print(f"Folder {player_folder_path} not found. Skipping.")

            except Exception as e:
                print(f"Error processing {first_name} {second_name} (ID: {player_id}) for season {season}: {e}")

    # Example usage
data_directory = "Fantasy-Premier-League/data"
players_to_remove = [
        {"season": "2020-21", "first_name": "Ben", "second_name": "Davies", "id": 653},
        {"season": "2021-22", "first_name": "Álvaro", "second_name": "Fernández", "id": 556},
        {"season": "2021-22", "first_name": "Álvaro", "second_name": "Fernández", "id": 728},
        {"season": "2023-24", "first_name": "Álvaro", "second_name": "Fernández Carreras", "id": 380},
        {"season": "2021-22", "first_name": "Ben", "second_name": "Davies", "id": 248},
        {"season": "2022-23", "first_name": "Ben", "second_name": "Davies", "id": 499},
]

remove_players_and_folders(data_directory, players_to_remove)

Removed Ben Davies (ID: 653) from Fantasy-Premier-League/data/2020-21/player_idlist.csv.
Removed folder: Fantasy-Premier-League/data/2020-21/players/Ben_Davies_653
Removed Álvaro Fernández (ID: 556) from Fantasy-Premier-League/data/2021-22/player_idlist.csv.
Removed folder: Fantasy-Premier-League/data/2021-22/players/Álvaro_Fernández_556
Removed Álvaro Fernández (ID: 728) from Fantasy-Premier-League/data/2021-22/player_idlist.csv.
Removed folder: Fantasy-Premier-League/data/2021-22/players/Álvaro_Fernández_728
Removed Álvaro Fernández Carreras (ID: 380) from Fantasy-Premier-League/data/2023-24/player_idlist.csv.
Removed folder: Fantasy-Premier-League/data/2023-24/players/Álvaro_Fernández Carreras_380
Removed Ben Davies (ID: 248) from Fantasy-Premier-League/data/2021-22/player_idlist.csv.
Removed folder: Fantasy-Premier-League/data/2021-22/players/Ben_Davies_248
Removed Ben Davies (ID: 499) from Fantasy-Premier-League/data/2022-23/player_idlist.csv.
Removed folder: Fantasy-Premier-Leagu

In [4]:
# First let's get each player position and id for the season
def process_season_data(data_directory, seasons):
    for season in seasons:
        season_path = os.path.join(data_directory, season)

        if os.path.isdir(season_path):
            player_idlist_path = os.path.join(season_path, "player_idlist.csv")
            cleaned_players_path = os.path.join(season_path, "cleaned_players.csv")
            output_file_path = os.path.join(season_path, "processed_players.csv")

            if os.path.exists(player_idlist_path) and os.path.exists(cleaned_players_path):
                try:
                    player_idlist_df = pd.read_csv(player_idlist_path)
                    cleaned_players_df = pd.read_csv(cleaned_players_path)

                    merged_df = pd.merge(
                        player_idlist_df[['id', 'first_name', 'second_name']],
                        cleaned_players_df[['first_name', 'second_name', 'element_type']],
                        on=['first_name', 'second_name'],
                        how='inner'
                    )

                    # Rename element_type to position
                    merged_df.rename(columns={'element_type': 'position'}, inplace=True)

                    # Save the DataFrame to a new file
                    merged_df.to_csv(output_file_path, index=False)
                    print(f"Processed and saved: {output_file_path}")
                except Exception as e:
                    print(f"Error processing season {season}: {e}")
            else:
                print(f"Missing required files in {season_path}: player_idlist.csv or cleaned_players.csv")

In [5]:
data_directory = "Fantasy-Premier-League/data"
seasons = ["2024-25", "2023-24", "2022-23", "2021-22", "2020-21"]
process_season_data(data_directory, seasons)

Processed and saved: Fantasy-Premier-League/data/2024-25/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2023-24/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2022-23/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2021-22/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2020-21/processed_players.csv


In [6]:
def merge_player_ids(data_directory, seasons, output_file="master_player_list.csv"):
    player_data = {}
    next_unique_id = 1
    
    # We want to base ids on 2024-25 season
    main_season = "2024-25"
    main_season_path = os.path.join(data_directory, main_season, "processed_players.csv")
    
    if os.path.exists(main_season_path):
        try:
            main_processed_df = pd.read_csv(main_season_path)
            
            for _, row in main_processed_df.iterrows():
                full_name = f"{row['first_name']} {row['second_name']}"
                if full_name not in player_data:
                    player_data[full_name] = {
                        "First_Name": row['first_name'],
                        "Last_Name": row['second_name'],
                        "Unique_ID": row['id']
                    }
                # Add the 24_id for the main season
                player_data[full_name]['24_id'] = row['id']
                next_unique_id = max(next_unique_id, row['id'] + 1)  # Ensure greater ids to avoid reusing

        except Exception as e:
            print(f"Error processing {main_season_path}: {e}")
    else:
        print(f"Missing processed_players.csv for season: {main_season}")

    # Process other seasons
    for season in seasons:
        
        if season != main_season:
            season_path = os.path.join(data_directory, season, "processed_players.csv")
            if os.path.exists(season_path):
                try:
                    processed_df = pd.read_csv(season_path)
                    
                    season_short = season[:4][-2:]
                    
                    for _, row in processed_df.iterrows():
                        full_name = f"{row['first_name']} {row['second_name']}"
                        if full_name not in player_data:
                            player_data[full_name] = {
                                "First_Name": row['first_name'],
                                "Last_Name": row['second_name'],
                                "Unique_ID": next_unique_id
                            }
                            next_unique_id += 1
                        # Add the season ID
                        player_data[full_name][f"{season_short}_id"] = row['id']
                except Exception as e:
                    print(f"Error processing {season_path}: {e}")

    consolidated_df = pd.DataFrame.from_dict(player_data, orient='index').reset_index(drop=True)
    
    # Ensure all ID columns are integers
    id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
    consolidated_df[id_columns] = consolidated_df[id_columns].fillna(-1).astype(int)
    
    # Save the consolidated DataFrame to a CSV
    output_path = os.path.join(data_directory, output_file)
    consolidated_df.to_csv(output_path, index=False)
    print(f"Consolidated player data saved to {output_path}")

    return consolidated_df

In [7]:
merge_player_ids(data_directory, seasons)

Consolidated player data saved to Fantasy-Premier-League/data/master_player_list.csv


,First_Name,Last_Name,Unique_ID,24_id,23_id,22_id,21_id,20_id
0,Fábio,Ferreira Vieira,1,1,4,25,-1,-1
1,Gabriel,Fernando de Jesus,2,2,8,28,263,282
2,Gabriel,dos Santos Magalhães,3,3,5,16,-1,-1
3,Kai,Havertz,4,4,6,145,141,500
4,Karl,Hein,5,5,646,655,532,-1
...,...,...,...,...,...,...,...,...
1746,Theo,Corbeanu,1747,-1,-1,-1,-1,610
1747,Taylor,Perry,1748,-1,-1,-1,-1,611
1748,Lewis,Richards,1749,-1,-1,-1,-1,615
1749,Nigel,Lonwijk,1750,-1,-1,-1,-1,629


In [8]:
# Script below was used to get duplicate candidates

# consolidated_file = "master_player_list.csv"
# consolidated_path = os.path.join(data_directory, consolidated_file)
# consolidated_df = pd.read_csv(consolidated_path)

# SIMILARITY_THRESHOLD = 80
# FIRST_NAME_SIMILARITY_THRESHOLD = 40

# def is_pair(player_a, player_b, id_columns):
#     # Check for ID overlaps across seasons
#     id_overlap = any(
#         player_a[col] != -1 and player_b[col] != -1
#         for col in id_columns
#     )
#     if id_overlap:
#         return False
    
#     # Now we check the names similarity
#     first_name_similarity = fuzz.partial_ratio(player_a["First_Name"], player_b["First_Name"])
#     if first_name_similarity <= FIRST_NAME_SIMILARITY_THRESHOLD:
#         return False
    
#     # Check if one last name is contained in the other
#     if player_a["Last_Name"] in player_b["Last_Name"] or player_b["Last_Name"] in player_a["Last_Name"]:
#         return True

#     # Calculate string similarity only if containment check fails
#     similarity = fuzz.partial_ratio(player_a["Last_Name"], player_b["Last_Name"])
#     return similarity >= SIMILARITY_THRESHOLD

# # Identify all ID columns
# id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
# pairs = []
# for i, player_a in consolidated_df.iterrows():
#     for j, player_b in consolidated_df.iterrows():
#         if i >= j:  # Avoid duplicate and self-pairs
#             continue
#         if is_pair(player_a, player_b, id_columns):
#             pairs.append({
#                 "Player_A_First_Name": player_a["First_Name"],
#                 "Player_A_Last_Name": player_a["Last_Name"],
#                 "Player_B_First_Name": player_b["First_Name"],
#                 "Player_B_Last_Name": player_b["Last_Name"],
#             })

# # Convert pairs to a DataFrame
# created_csv_dir = "created_csv"
# pairs_df = pd.DataFrame(pairs)
# if not pairs_df.empty:
#     output_path = os.path.join(created_csv_dir, "players_with_potential_renames.csv")
#     pairs_df.to_csv(output_path, index=False)
#     print(f"Potential renamed players saved to {output_path}")
# else:
#     print("No potential renamed players found.")

In [9]:
# After manual checking the candidates now we have to update master_player_list.csv 
# (new file will me master_player_verified)
verified_renames = pd.read_csv("created_csv/verified_renames.csv")
master_player_file = "master_player_list.csv"
master_player_path = os.path.join(data_directory, master_player_file)
master_player_list = pd.read_csv(master_player_path)

# Player A ID will be smaller thanks to the way the duplicate finding was implemented
for _, row in verified_renames.iterrows():
    player_a_first = row["Player_A_First_Name"]
    player_a_last = row["Player_A_Last_Name"]
    player_b_first = row["Player_B_First_Name"]
    player_b_last = row["Player_B_Last_Name"]
    
    # Find rows for Player A and Player B in the master_player_list
    player_a_row = master_player_list[
        (master_player_list["First_Name"] == player_a_first) &
        (master_player_list["Last_Name"] == player_a_last)
    ]

    player_b_row = master_player_list[
        (master_player_list["First_Name"] == player_b_first) &
        (master_player_list["Last_Name"] == player_b_last)
    ]

    # Ensure both rows exist
    if player_a_row.empty or player_b_row.empty:
        print(f"Warning: Missing player in master_player_list. Player A: {player_a_first} {player_a_last}, Player B: {player_b_first} {player_b_last}")
        continue

    player_a_id = player_a_row["Unique_ID"].iloc[0]
    player_b_id = player_b_row["Unique_ID"].iloc[0]
    
    # Update the smaller ID row with non-negative season data from the larger ID row
    for season in ["24_id", "23_id", "22_id", "21_id", "20_id"]:
        if player_b_row[season].iloc[0] != -1:
            master_player_list.loc[master_player_list["Unique_ID"] == player_a_id, season] = player_b_row[season].iloc[0]

    # Remove player B
    master_player_list = master_player_list[master_player_list["Unique_ID"] != player_b_id]

# Save the updated master player list to a new file
master_player_list.to_csv("Fantasy-Premier-League/data/master_player_verified.csv", index=False)

In [10]:
def update_processed_players_and_rename_folders(data_directory, seasons, master_player_file):
    master_player_path = os.path.join(data_directory, master_player_file)
    master_player_verified = pd.read_csv(master_player_path)

    for season in seasons:
        season_path = os.path.join(data_directory, season)
        players_folder_path = os.path.join(season_path, "players")
        processed_file = os.path.join(season_path, "processed_players.csv")
        verified_file = os.path.join(season_path, "processed_players_verified.csv")
        
        if season == "2024-25":
            print(f"Processing special case: {season}")

            if os.path.exists(processed_file):
                try:
                    # Simply copy processed_players.csv to processed_players_verified.csv
                    shutil.copy(processed_file, verified_file)
                    print(f"Copied {processed_file} to {verified_file}")
                except Exception as e:
                    print(f"Error copying {processed_file} to {verified_file}: {e}")
            else:
                print(f"Processed players file not found for season: {season}")
            continue

        season_short = season[:4][-2:]  # Extract season short (e.g., "21", "22")
        season_id_column = f"{season_short}_id"

        if os.path.exists(processed_file):
            print(f"Processing season: {season}")
            processed_players = pd.read_csv(processed_file)

            # Get all player folders in the players folder
            player_folders = []
            if os.path.exists(players_folder_path):
                player_folders = [
                    f for f in os.listdir(players_folder_path)
                    if os.path.isdir(os.path.join(players_folder_path, f))
                ]

            # Update IDs and rename folders
            for index, row in processed_players.iterrows():
                current_id = row["id"]
                # Match the season-specific ID in master_player_verified
                matching_row = master_player_verified[master_player_verified[season_id_column] == current_id]

                if not matching_row.empty:
                    new_data = matching_row.iloc[0]
                    new_unique_id = new_data["Unique_ID"]
                    new_first_name = new_data["First_Name"]
                    new_last_name = new_data["Last_Name"]

                    # Update the processed_players row
                    processed_players.at[index, "id"] = new_unique_id
                    processed_players.at[index, "first_name"] = new_first_name
                    processed_players.at[index, "second_name"] = new_last_name

                    # Find and rename the player's folder
                    matching_folder = next(
                        (f for f in player_folders if f.endswith(f"_{current_id}")), None
                    )
                    if matching_folder:
                        old_path = os.path.join(players_folder_path, matching_folder)
                        new_folder_name = f"{new_first_name}_{new_last_name}_{new_unique_id}"
                        new_path = os.path.join(players_folder_path, new_folder_name)

                        # Ensure destination directory does not already exist
                        if os.path.exists(new_path):
                            continue
                        try:
                            shutil.move(old_path, new_path)
                            print(f"Successfully renamed folder {matching_folder} to {new_folder_name}")
                            player_folders.remove(matching_folder)
                        except Exception as e:
                            print(f"Error moving {old_path} to {new_path}: {e}")

            # Save the updated processed_players file
            processed_players.to_csv(verified_file, index=False)
            print(f"Updated file and folders saved for season: {season}")
        else:
            print(f"Processed players file not found for season: {season}")

In [11]:
master_player_file = "master_player_verified.csv"

update_processed_players_and_rename_folders(data_directory, seasons, master_player_file)

Processing special case: 2024-25
Copied Fantasy-Premier-League/data/2024-25/processed_players.csv to Fantasy-Premier-League/data/2024-25/processed_players_verified.csv
Processing season: 2023-24
Successfully renamed folder Folarin_Balogun_1 to Folarin_Balogun_675
Successfully renamed folder Cédric_Alves Soares_2 to Cédric_Alves Soares_676
Successfully renamed folder Mohamed_Elneny_3 to Mohamed_Elneny_677
Successfully renamed folder Fábio_Ferreira Vieira_4 to Fábio_Ferreira Vieira_1
Successfully renamed folder Gabriel_dos Santos Magalhães_5 to Gabriel_dos Santos Magalhães_3
Successfully renamed folder Kai_Havertz_6 to Kai_Havertz_4
Successfully renamed folder Gabriel_Fernando de Jesus_8 to Gabriel_Fernando de Jesus_2
Successfully renamed folder Jorge Luiz_Frello Filho_9 to Jorge Luiz_Frello Filho_7
Successfully renamed folder Jakub_Kiwior_10 to Jakub_Kiwior_8
Successfully renamed folder Marcus_Oliveira Alencar_11 to Marcus_Oliveira Alencar_678
Successfully renamed folder Gabriel_Martine

Successfully renamed folder Arnaut_Danjuma Groeneveld_601 to Arnaut_Danjuma Groeneveld_834
Successfully renamed folder Lewis_Dobbin_624 to Lewis_Dobbin_37
Successfully renamed folder Youssef_Ramalho Chermiti_645 to Youssef_Ramalho Chermiti_237
Successfully renamed folder Thomas_Cannon_649 to Tom_Cannon_286
Successfully renamed folder Tyler_Onyango_650 to Tyler_Onyango_836
Successfully renamed folder Jack_Harrison_661 to Jack_Harrison_224
Successfully renamed folder Norberto Bercique_Gomes Betuncal_691 to Norberto Bercique_Gomes Betuncal_218
Successfully renamed folder Mackenzie_Hunt_719 to Mackenzie_Hunt_837
Successfully renamed folder Jenson_Metcalfe_760 to Jenson_Metcalfe_614
Successfully renamed folder Elijah_Campbell_761 to Elijah_Campbell_838
Successfully renamed folder Lewis_Warrington_841 to Lewis_Warrington_839
Successfully renamed folder Billy_Crellin_863 to Billy_Crellin_840
Successfully renamed folder Armando_Broja_193 to Armando_Broja_156
Successfully renamed folder Alex_Iw

Successfully renamed folder Oliver_Scarles_784 to Oliver_Scarles_1051
Successfully renamed folder Callum_Marshall_785 to Callum_Marshall_1052
Successfully renamed folder George_Earthy_829 to George_Earthy_519
Successfully renamed folder Tommy_Doyle_351 to Tommy_Doyle_544
Successfully renamed folder Rayan_Aït-Nouri_545 to Rayan_Aït-Nouri_533
Successfully renamed folder Daniel_Bentley_546 to Daniel_Bentley_536
Successfully renamed folder Bendegúz_Bolla_547 to Bendegúz_Bolla_1053
Successfully renamed folder Hugo_Bueno López_548 to Hugo_Bueno López_549
Successfully renamed folder Francisco Jorge_Tomás Oliveira_549 to Francisco Jorge_Tomás Oliveira_537
Successfully renamed folder Luke_Cundle_550 to Luke_Cundle_540
Successfully renamed folder Craig_Dawson_551 to Craig_Dawson_542
Successfully renamed folder Fábio_Silva_552 to Fábio_Silva_545
Successfully renamed folder Gonçalo Manuel_Ganchinho Guedes_554 to Gonçalo Manuel_Ganchinho Guedes_548
Successfully renamed folder Joe_Hodge_555 to Joe_H

Successfully renamed folder Ivan_Neves Abreu Cavaleiro_214 to Ivan_Neves Abreu Cavaleiro_841
Successfully renamed folder Kenny_Tete_215 to Kenny_Tete_257
Successfully renamed folder Josh_Onomah_216 to Josh_Onomah_1137
Successfully renamed folder Antonee_Robinson_217 to Antonee_Robinson_255
Successfully renamed folder Rodrigo_Muniz Carvalho_218 to Rodrigo_Muniz Carvalho_251
Successfully renamed folder Jay_Stansfield_219 to Jay_Stansfield_256
Successfully renamed folder João_Palhinha Gonçalves_220 to João_Palhinha Gonçalves_846
Successfully renamed folder Daniel_James_231 to Daniel_James_1138
Successfully renamed folder Andreas_Hoelgebaum Pereira_346 to Andreas_Hoelgebaum Pereira_240
Successfully renamed folder Issa_Diop_470 to Issa_Diop_245
Successfully renamed folder Tyrese_Francois_521 to Tyrese_Francois_842
Successfully renamed folder Kevin_Mbabu_532 to Kevin_Mbabu_250
Successfully renamed folder Luke_Harris_546 to Luke_Harris_246
Successfully renamed folder Manor_Solomon_562 to Mano

Successfully renamed folder Nathan_Fraser_657 to Nathan_Fraser_546
Successfully renamed folder Harvey_Griffiths_658 to Harvey_Griffiths_1062
Successfully renamed folder Dexter_Lembikisa_664 to Dexter_Lembikisa_1279
Successfully renamed folder Matheus_Santos Carneiro Da Cunha_682 to Matheus_Santos Carneiro Da Cunha_541
Successfully renamed folder Mario_Lemina_698 to Mario_Lemina_557
Successfully renamed folder Pablo_Sarabia_704 to Pablo_Sarabia_565
Successfully renamed folder Daniel_Bentley_717 to Daniel_Bentley_536
Successfully renamed folder João Victor_Gomes da Silva_721 to João Victor_Gomes da Silva_553
Updated file and folders saved for season: 2022-23
Processing season: 2021-22
Successfully renamed folder Bernd_Leno_1 to Bernd_Leno_248
Successfully renamed folder Rúnar Alex_Rúnarsson_2 to Rúnar Alex_Rúnarsson_680
Successfully renamed folder Willian_Borges Da Silva_3 to Willian_Borges da Silva_848
Successfully renamed folder Pierre-Emerick_Aubameyang_4 to Pierre-Emerick_Aubameyang_

Successfully renamed folder Fabio Henrique_Tavares_232 to Fabio Henrique_Tavares_857
Successfully renamed folder Mohamed_Salah_233 to Mohamed_Salah_328
Successfully renamed folder Andrew_Robertson_234 to Andrew_Robertson_335
Successfully renamed folder Divock_Origi_235 to Divock_Origi_984
Successfully renamed folder Takumi_Minamino_236 to Takumi_Minamino_1394
Successfully renamed folder Trent_Alexander-Arnold_237 to Trent_Alexander-Arnold_311
Successfully renamed folder Joseph_Gomez_238 to Joe_Gomez_322
Successfully renamed folder Naby_Keita_239 to Naby_Keita_1197
Successfully renamed folder Diogo_Jota_240 to Diogo_Jota_1395
Successfully renamed folder Nathaniel_Phillips_241 to Nathaniel_Phillips_332
Successfully renamed folder Caoimhin_Kelleher_242 to Caoimhin_Kelleher_325
Successfully renamed folder Ibrahima_Konaté_243 to Ibrahima_Konaté_326
Successfully renamed folder Curtis_Jones_244 to Curtis_Jones_324
Successfully renamed folder Konstantinos_Tsimikas_245 to Konstantinos_Tsimikas_

Successfully renamed folder Wesley_Moraes_48 to Wesley_Moraes Ferreira da Silva_696
Successfully renamed folder Mbwana_Samatta_49 to Mbwana Ally_Samatta_1310
Successfully renamed folder Keinan_Davis_50 to Keinan_Davis_691
Successfully renamed folder Indiana_Vassilev_51 to Indiana_Vassilev_1563
Successfully renamed folder Douglas Luiz_Soares de Paulo_52 to Douglas Luiz_Soares de Paulo_692
Successfully renamed folder Ross_Barkley_106 to Ross_Barkley_29
Successfully renamed folder Matthew_Cash_496 to Matty_Cash_32
Successfully renamed folder Ollie_Watkins_514 to Ollie_Watkins_58
Successfully renamed folder Bertrand_Traoré_540 to Bertrand_Traoré_695
Successfully renamed folder Jacob_Ramsey_554 to Jacob_Ramsey_53
Successfully renamed folder Morgan_Sanson_647 to Morgan_Sanson_694
Successfully renamed folder Kaine_Hayden_671 to Kaine_Kesler-Hayden_43
Successfully renamed folder Carney_Chukwuemeka_672 to Carney_Chukwuemeka_161
Successfully renamed folder Jaden_Philogene-Bidace_683 to Jaden_Phi

Successfully renamed folder Rolando_Aarons_337 to Rolando_Aarons_1663
Successfully renamed folder Allan_Saint-Maximin_338 to Allan_Saint-Maximin_952
Successfully renamed folder Miguel_Almirón_339 to Miguel_Almirón Rejala_391
Successfully renamed folder Joelinton Cássio_Apolinário de Lira_340 to Joelinton Cássio_Apolinário de Lira_403
Successfully renamed folder Yoshinori_Muto_341 to Yoshinori_Muto_1664
Successfully renamed folder Kelland_Watts_342 to Kell_Watts_953
Successfully renamed folder Sean_Longstaff_480 to Sean_Longstaff_410
Successfully renamed folder Jeff_Hendrick_485 to Jeff_Hendrick_949
Successfully renamed folder Matthew_Longstaff_487 to Matthew_Longstaff_1423
Successfully renamed folder Mark_Gillespie_488 to Mark_Gillespie_948
Successfully renamed folder Jacob_Murphy_497 to Jacob_Murphy_402
Successfully renamed folder Callum_Wilson_506 to Callum_Wilson_421
Successfully renamed folder Ryan_Fraser_507 to Ryan_Fraser_397
Successfully renamed folder Jamal_Lewis_513 to Jamal_L

In [12]:
# Now we will start manipulating player's data. First let's add to each player row his position
# (1 - GK, 2 - DEF, 3 - MID, 4 - FWD) and add it to the folder's name as well
# Let's also add the Full Name of the player


def update_player_folders_and_gw(season_path):
    position_mapping = {"GK": 1, "DEF": 2, "MID": 3, "FWD": 4}
    
    processed_players_path = os.path.join(season_path, "processed_players_verified.csv")
    players_folder_path = os.path.join(season_path, "players")
    
    if not os.path.exists(processed_players_path):
        print(f"{processed_players_path} not found.")
        return
    
    processed_players = pd.read_csv(processed_players_path)
    
    for _, row in processed_players.iterrows():
        player_id = row['id']
        first_name = row['first_name']
        second_name = row['second_name']
        position_str = row['position']
        
        pos_numeric = position_mapping.get(position_str, 0)  # Default to 0 if unknown
        if pos_numeric == 0:
            print(f"Unknown position for player {first_name} {second_name} (ID: {player_id}). Skipping.")
            continue
        
        old_folder_name = f"{first_name}_{second_name}_{player_id}"
        old_folder_path = os.path.join(players_folder_path, old_folder_name)
        if not os.path.exists(old_folder_path):
            print(f"Folder {old_folder_name} not found. Skipping.")
            continue

        new_folder_name = f"{position_str}_{old_folder_name}"
        new_folder_path = os.path.join(players_folder_path, new_folder_name)
        
        try:
            shutil.move(old_folder_path, new_folder_path)
            print(f"Renamed folder: {old_folder_name} -> {new_folder_name}")
        except Exception as e:
            print(f"Error renaming {old_folder_name} to {new_folder_name}: {e}")
            continue
        
        gw_file_path = os.path.join(new_folder_path, "gw.csv")
        
        if os.path.exists(gw_file_path):
            try:
                gw_data = pd.read_csv(gw_file_path)
                gw_data['POS'] = pos_numeric
                gw_data['Full Name'] = f"{first_name} {second_name}"
                gw_data.to_csv(gw_file_path, index=False)
                print(f"Updated POS column in {gw_file_path}")
            except Exception as e:
                print(f"Error updating {gw_file_path}: {e}")

In [13]:
season_paths = [
    "Fantasy-Premier-League/data/2020-21",
    "Fantasy-Premier-League/data/2021-22",
    "Fantasy-Premier-League/data/2022-23",
    "Fantasy-Premier-League/data/2023-24",
    "Fantasy-Premier-League/data/2024-25"
]

for season_path in season_paths:
    print(f"Processing season: {season_path}")
    update_player_folders_and_gw(season_path)

Processing season: Fantasy-Premier-League/data/2020-21
Renamed folder: Mesut_Özil_1550 -> MID_Mesut_Özil_1550
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Mesut_Özil_1550/gw.csv
Renamed folder: Sokratis_Papastathopoulos_1551 -> DEF_Sokratis_Papastathopoulos_1551
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Sokratis_Papastathopoulos_1551/gw.csv
Renamed folder: David_Luiz Moreira Marinho_1552 -> DEF_David_Luiz Moreira Marinho_1552
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_David_Luiz Moreira Marinho_1552/gw.csv
Renamed folder: Pierre-Emerick_Aubameyang_791 -> MID_Pierre-Emerick_Aubameyang_791
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Pierre-Emerick_Aubameyang_791/gw.csv
Renamed folder: Cédric_Alves Soares_676 -> DEF_Cédric_Alves Soares_676
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Cédric_Alves Soares_676/gw.csv
Renamed folder: Alexandre_Lacazette_1282 -> FW

Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Ross_Barkley_29/gw.csv
Renamed folder: Matty_Cash_32 -> DEF_Matty_Cash_32
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Matty_Cash_32/gw.csv
Renamed folder: Ollie_Watkins_58 -> FWD_Ollie_Watkins_58
Updated POS column in Fantasy-Premier-League/data/2020-21/players/FWD_Ollie_Watkins_58/gw.csv
Renamed folder: Bertrand_Traoré_695 -> MID_Bertrand_Traoré_695
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Bertrand_Traoré_695/gw.csv
Renamed folder: Jacob_Ramsey_53 -> MID_Jacob_Ramsey_53
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Jacob_Ramsey_53/gw.csv
Renamed folder: Morgan_Sanson_694 -> MID_Morgan_Sanson_694
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Morgan_Sanson_694/gw.csv
Renamed folder: Kaine_Kesler-Hayden_43 -> MID_Kaine_Kesler-Hayden_43
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Kaine_Kesler-

Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Anthony_Driscoll-Glennon_1578/gw.csv
Renamed folder: Mace_Goodridge_1579 -> MID_Mace_Goodridge_1579
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Mace_Goodridge_1579/gw.csv
Renamed folder: Will_Norris_1346 -> GK_Will_Norris_1346
Updated POS column in Fantasy-Premier-League/data/2020-21/players/GK_Will_Norris_1346/gw.csv
Renamed folder: Max_Thompson_1580 -> FWD_Max_Thompson_1580
Updated POS column in Fantasy-Premier-League/data/2020-21/players/FWD_Max_Thompson_1580/gw.csv
Renamed folder: Lewis_Richardson_1348 -> FWD_Lewis_Richardson_1348
Updated POS column in Fantasy-Premier-League/data/2020-21/players/FWD_Lewis_Richardson_1348/gw.csv
Renamed folder: Anthony_Mancini_1352 -> MID_Anthony_Mancini_1352
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Anthony_Mancini_1352/gw.csv
Renamed folder: Joel_Mumbongo_1581 -> FWD_Joel_Mumbongo_1581
Updated POS column in Fantasy-Premier-Le

Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Alex_Iwobi_247/gw.csv
Renamed folder: Jonjoe_Kenny_1373 -> DEF_Jonjoe_Kenny_1373
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Jonjoe_Kenny_1373/gw.csv
Renamed folder: Jean-Philippe_Gbamin_829 -> MID_Jean-Philippe_Gbamin_829
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Jean-Philippe_Gbamin_829/gw.csv
Renamed folder: Yerry_Mina_1123 -> DEF_Yerry_Mina_1123
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Yerry_Mina_1123/gw.csv
Renamed folder: Tom_Davies_1016 -> MID_Tom_Davies_1016
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Tom_Davies_1016/gw.csv
Renamed folder: Dominic_Calvert-Lewin_220 -> FWD_Dominic_Calvert-Lewin_220
Updated POS column in Fantasy-Premier-League/data/2020-21/players/FWD_Dominic_Calvert-Lewin_220/gw.csv
Renamed folder: Mason_Holgate_225 -> DEF_Mason_Holgate_225
Updated POS column in Fantasy-Premier-League/d

Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Wes_Morgan_1617/gw.csv
Renamed folder: Kasper_Schmeichel_1145 -> GK_Kasper_Schmeichel_1145
Updated POS column in Fantasy-Premier-League/data/2020-21/players/GK_Kasper_Schmeichel_1145/gw.csv
Renamed folder: Christian_Fuchs_1618 -> DEF_Christian_Fuchs_1618
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Christian_Fuchs_1618/gw.csv
Renamed folder: Jonny_Evans_371 -> DEF_Jonny_Evans_371
Updated POS column in Fantasy-Premier-League/data/2020-21/players/DEF_Jonny_Evans_371/gw.csv
Renamed folder: Adrien_Silva_1619 -> MID_Adrien_Silva_1619
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Adrien_Silva_1619/gw.csv
Renamed folder: Marc_Albrighton_1147 -> MID_Marc_Albrighton_1147
Updated POS column in Fantasy-Premier-League/data/2020-21/players/MID_Marc_Albrighton_1147/gw.csv
Renamed folder: Matty_James_1620 -> MID_Matty_James_1620
Updated POS column in Fantasy-Premier-League/data/2020-

...

Renamed folder: Calvin_Bassey_241 -> DEF_Calvin_Bassey_241
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Calvin_Bassey_241/gw.csv
Renamed folder: Steven_Benda_242 -> GK_Steven_Benda_242
Updated POS column in Fantasy-Premier-League/data/2024-25/players/GK_Steven_Benda_242/gw.csv
Renamed folder: Tom_Cairney_243 -> MID_Tom_Cairney_243
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Tom_Cairney_243/gw.csv
Renamed folder: Timothy_Castagne_244 -> DEF_Timothy_Castagne_244
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Timothy_Castagne_244/gw.csv
Renamed folder: Issa_Diop_245 -> DEF_Issa_Diop_245
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Issa_Diop_245/gw.csv
Renamed folder: Luke_Harris_246 -> MID_Luke_Harris_246
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Luke_Harris_246/gw.csv
Renamed folder: Alex_Iwobi_247 -> MID_Alex_Iwobi_247
Updated POS column in Fantasy-Premier-Leagu

Renamed folder: Ben_Nelson_616 -> DEF_Ben_Nelson_616
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Ben_Nelson_616/gw.csv
Renamed folder: Bilal_El Khannouss_635 -> MID_Bilal_El Khannouss_635
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Bilal_El Khannouss_635/gw.csv
Renamed folder: Alisson_Ramses Becker_310 -> GK_Alisson_Ramses Becker_310
Updated POS column in Fantasy-Premier-League/data/2024-25/players/GK_Alisson_Ramses Becker_310/gw.csv
Renamed folder: Trent_Alexander-Arnold_311 -> DEF_Trent_Alexander-Arnold_311
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Trent_Alexander-Arnold_311/gw.csv
Renamed folder: Stefan_Bajčetić Maquieira_312 -> MID_Stefan_Bajčetić Maquieira_312
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Stefan_Bajčetić Maquieira_312/gw.csv
Renamed folder: Conor_Bradley_313 -> DEF_Conor_Bradley_313
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Conor_Brad

Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Luke_Shaw_387/gw.csv
Renamed folder: Joshua_Zirkzee_389 -> FWD_Joshua_Zirkzee_389
Updated POS column in Fantasy-Premier-League/data/2024-25/players/FWD_Joshua_Zirkzee_389/gw.csv
Renamed folder: Leny_Yoro_572 -> DEF_Leny_Yoro_572
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Leny_Yoro_572/gw.csv
Renamed folder: Matthijs_de Ligt_593 -> DEF_Matthijs_de Ligt_593
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Matthijs_de Ligt_593/gw.csv
Renamed folder: Noussair_Mazraoui_594 -> DEF_Noussair_Mazraoui_594
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Noussair_Mazraoui_594/gw.csv
Renamed folder: Toby_Collyer_613 -> MID_Toby_Collyer_613
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Toby_Collyer_613/gw.csv
Renamed folder: Ethan_Wheatley_648 -> FWD_Ethan_Wheatley_648
Updated POS column in Fantasy-Premier-League/data/2024-25/players/FWD

Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Shea_Charles_458/gw.csv
Renamed folder: Samuel_Edozie_459 -> MID_Samuel_Edozie_459
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Samuel_Edozie_459/gw.csv
Renamed folder: Ronnie_Edwards_460 -> DEF_Ronnie_Edwards_460
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Ronnie_Edwards_460/gw.csv
Renamed folder: Taylor_Harwood-Bellis_461 -> DEF_Taylor_Harwood-Bellis_461
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Taylor_Harwood-Bellis_461/gw.csv
Renamed folder: Kamaldeen_Sulemana_462 -> MID_Kamaldeen_Sulemana_462
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Kamaldeen_Sulemana_462/gw.csv
Renamed folder: Adam_Lallana_463 -> MID_Adam_Lallana_463
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Adam_Lallana_463/gw.csv
Renamed folder: Juan_Larios López_464 -> DEF_Juan_Larios López_464
Updated POS column in Fantasy-

Updated POS column in Fantasy-Premier-League/data/2024-25/players/GK_Sam_Johnstone_204/gw.csv
Renamed folder: Rayan_Aït-Nouri_533 -> DEF_Rayan_Aït-Nouri_533
Updated POS column in Fantasy-Premier-League/data/2024-25/players/DEF_Rayan_Aït-Nouri_533/gw.csv
Renamed folder: Boubacar_Traoré_534 -> MID_Boubacar_Traoré_534
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Boubacar_Traoré_534/gw.csv
Renamed folder: Jean-Ricner_Bellegarde_535 -> MID_Jean-Ricner_Bellegarde_535
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Jean-Ricner_Bellegarde_535/gw.csv
Renamed folder: Daniel_Bentley_536 -> GK_Daniel_Bentley_536
Updated POS column in Fantasy-Premier-League/data/2024-25/players/GK_Daniel_Bentley_536/gw.csv
Renamed folder: Francisco Jorge_Tomás Oliveira_537 -> MID_Francisco Jorge_Tomás Oliveira_537
Updated POS column in Fantasy-Premier-League/data/2024-25/players/MID_Francisco Jorge_Tomás Oliveira_537/gw.csv
Renamed folder: Tawanda_Chirewa_538 -> MID_Ta

In [4]:
# Player data doesn't have his own team id (and they can switch teams during season so we have to add that)
def add_own_team_to_players(data_directory, season, fixtures_file):
    fixtures_path = os.path.join(data_directory, season, fixtures_file)
    if not os.path.exists(fixtures_path):
        print(f"Fixtures file not found for season {season}. Skipping.")
        return
    fixtures = pd.read_csv(fixtures_path)
    players_folder = os.path.join(data_directory, season, "players")
    
    if not os.path.exists(players_folder):
        print(f"Players folder not found for season {season}. Skipping.")
        return
    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        if os.path.isdir(player_folder_path):
            gw_file_path = os.path.join(player_folder_path, "gw.csv")
            
            if os.path.exists(gw_file_path):
                try:
                    gw_data = pd.read_csv(gw_file_path)

                    # Merge fixtures data to get team_a and team_h
                    gw_data = gw_data.merge(
                        fixtures[['id', 'team_a', 'team_h']],
                        left_on='fixture',
                        right_on='id',
                        how='left'
                    )

                    # Determine own_team based on was_home
                    gw_data['own_team'] = gw_data.apply(
                        lambda x: x['team_h'] if x['was_home'] else x['team_a'], axis=1
                    )

                    # Drop unnecessary columns (like the merged fixture id)
                    gw_data.drop(columns=['id', 'team_a', 'team_h'], inplace=True)

                    # Save the updated gw.csv
                    gw_data.to_csv(gw_file_path, index=False)
                    print(f"Updated {gw_file_path} with own_team.")
                except Exception as e:
                    print(f"Error processing {gw_file_path}: {e}")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"]
fixtures_file = "fixtures.csv"

for season in seasons:
    print(f"Processing season: {season}")
    add_own_team_to_players(data_directory, season, fixtures_file)

Processing season: 2020-21
Updated Fantasy-Premier-League/data/2020-21/players/MID_Gareth_Bale_1694/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ainsley_Maitland-Niles_1231/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Nathan_Redmond_780/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jacob_Ramsey_53/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jensen_Weir_1575/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Michy_Batshuayi_1354/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Ashley_Westwood_1342/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Chris_Smalling_1652/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ryan_Bertrand_1146/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Japhet_Tanganga_1034/gw.csv with own_

Updated Fantasy-Premier-League/data/2020-21/players/FWD_Dane_Scarlett_500/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Matija_Sarkic_1742/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jordan_Henderson_858/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Thiago_Alcántara do Nascimento_860/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Sam_Field_1713/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Chris_Basham_994/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Charlie_Taylor_475/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Joe_Hart_1467/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Shane_Flynn_1626/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Léo_Bonatini_1739/gw.csv with own_team.
Updated Fantasy-Premier-Leagu

Updated Fantasy-Premier-League/data/2020-21/players/MID_Martin_Ødegaard_13/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Diogo_Jota_1395/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Adama_Traoré_239/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Bernard_Anício Caldeira Duarte_1369/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Percy_Tau_1571/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Jed_Steer_1303/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Georginio_Wijnaldum_1639/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Michał_Karbownik_740/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ryan_Bennett_1738/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Frankie_Maguire_1680/gw.csv with own_team.
Updated Fantasy-Pr

Updated Fantasy-Premier-League/data/2020-21/players/MID_Carney_Chukwuemeka_161/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Wilfried_Zaha_1113/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_George_Baldock_993/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Liam_Hughes_1642/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Robert_Snodgrass_1715/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Jonjoe_Kenny_1373/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Jean-Philippe_Mateta_207/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Nicolas_Pépé_679/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Roberto_Jimenez Gago_1730/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Bernardo_Fernandes Da Silva Junior_1328/gw.csv with own_te

Updated Fantasy-Premier-League/data/2020-21/players/DEF_Aymeric_Laporte_919/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Charlie_Austin_1707/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Mateo_Kovacic_1100/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Richarlison_de Andrade_497/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jake_Livermore_1701/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Robin_Koch_1169/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Kieran_Tierney_21/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Morgan_Sanson_694/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Alex_Iwobi_247/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Conor_Coady_288/gw.csv with own_team.
Updated Fantasy-Premier-League/

...

Updated Fantasy-Premier-League/data/2024-25/players/FWD_Jamie_Vardy_306/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Kaden_Rodney_647/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Roman_Dixon_627/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Ola_Aina_422/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Christian_Eriksen_370/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Steven_Benda_242/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jack_Clarke_631/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Mads_Roerslev Rasmussen_105/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Joe_Worrall_448/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Ashley_Phillips_478/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024

Updated Fantasy-Premier-League/data/2024-25/players/FWD_Taiwo_Awoniyi_424/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Remi_Matthews_209/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Carlos Roberto_Forbs Borges_655/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Altay_Bayindir_367/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_João Pedro_Junqueira de Jesus_129/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Will_Hughes_203/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Isaac_Hayden_400/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Luke_Harris_246/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nayef_Aguerd_529/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_James_Ward-Prowse_531/gw.csv with own_team.
Updated Fanta

Updated Fantasy-Premier-League/data/2024-25/players/MID_Stephy_Mavididi_298/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Darwin_Núñez Ribeiro_316/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Emmanuel_Dennis_430/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ibrahim_Osman_127/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jorge_Cuenca Barreno_586/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Gonçalo Manuel_Ganchinho Guedes_548/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Sven_Botman_393/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jayden_Meghoma_649/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Matt_Targett_416/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jan_Bednarek_455/gw.csv with own_team.
Updated

Updated Fantasy-Premier-League/data/2024-25/players/MID_Ryan_Gravenberch_323/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jack_Stephens_472/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Aaron_Hickey_93/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Alexis_Mac Allister_329/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Bobby_Clark_315/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Issa_Diop_245/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Matt_O'Riley_629/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Karl_Hein_5/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Bruno_Guimarães Rodriguez Moura_394/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Will_Lankshear_609/gw.csv with own_team.
Updated Fantasy-Premier-Leag

Updated Fantasy-Premier-League/data/2024-25/players/MID_Nathan_Broadhead_262/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Mikey_Moore_610/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Max_Aarons_59/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Trent_Alexander-Arnold_311/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Carlos_Alcaraz Durán_450/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Marcus_Myers-Harness_271/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Chiedozie_Ogbene_632/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Oliwier_Zych_641/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jahmai_Simpson-Pusey_672/gw.csv with own_team.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Tariq_Lamptey_131/gw.csv with own_team.
Updated Fant

In [4]:
# Now we can add short team names for player and opponent and also strengths of both teams from teams.csv
def add_team_strengths_and_short_names(data_directory, season, teams_file):
    teams_path = os.path.join(data_directory, season, teams_file)
    
    if not os.path.exists(teams_path):
        print(f"Teams file not found for season {season}. Skipping.")
        return
    
    teams_data = pd.read_csv(teams_path).set_index('id')

    players_folder = os.path.join(data_directory, season, "players")
    if not os.path.exists(players_folder):
        print(f"Players folder not found for season {season}. Skipping.")
        return

    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")
        
        if os.path.exists(gw_file_path):
            try:
                gw_data = pd.read_csv(gw_file_path)

                # Merge opponent team strengths and short name
                gw_data = gw_data.merge(
                    teams_data[['short_name', 'strength_attack_home', 'strength_attack_away', 
                                'strength_defence_home', 'strength_defence_away']],
                    left_on='opponent_team',
                    right_index=True,
                    how='left'
                ).rename(columns={
                    'short_name': 'opponent_short_name',
                    'strength_attack_home': 'opponent_attack_home',
                    'strength_attack_away': 'opponent_attack_away',
                    'strength_defence_home': 'opponent_defence_home',
                    'strength_defence_away': 'opponent_defence_away'
                })

                # Add opponent strengths based on was_home
                gw_data['opponent_attack'] = gw_data.apply(
                    lambda x: x['opponent_attack_away'] if x['was_home'] else x['opponent_attack_home'], axis=1
                )
                gw_data['opponent_defense'] = gw_data.apply(
                    lambda x: x['opponent_defence_away'] if x['was_home'] else x['opponent_defence_home'], axis=1
                )

                # Merge own team strengths and short name
                gw_data = gw_data.merge(
                    teams_data[['short_name', 'strength_attack_home', 'strength_attack_away', 
                                'strength_defence_home', 'strength_defence_away']],
                    left_on='own_team',
                    right_index=True,
                    how='left'
                ).rename(columns={
                    'short_name': 'own_short_name',
                    'strength_attack_home': 'own_attack_home',
                    'strength_attack_away': 'own_attack_away',
                    'strength_defence_home': 'own_defence_home',
                    'strength_defence_away': 'own_defence_away'
                })

                # Add own team strengths based on was_home
                gw_data['own_attack'] = gw_data.apply(
                    lambda x: x['own_attack_home'] if x['was_home'] else x['own_attack_away'], axis=1
                )
                gw_data['own_defense'] = gw_data.apply(
                    lambda x: x['own_defence_home'] if x['was_home'] else x['own_defence_away'], axis=1
                )

                # Drop temporary columns to keep only relevant data
                gw_data.drop(columns=[
                    'opponent_attack_home', 'opponent_attack_away',
                    'opponent_defence_home', 'opponent_defence_away',
                    'own_attack_home', 'own_attack_away',
                    'own_defence_home', 'own_defence_away'
                ], inplace=True)

                # Save the updated gw.csv
                gw_data.to_csv(gw_file_path, index=False)
                print(f"Updated {gw_file_path} with team strengths and short names.")
            except Exception as e:
                print(f"Error processing {gw_file_path}: {e}")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"]
teams_file = "teams.csv"

for season in seasons:
    print(f"Processing season: {season}")
    add_team_strengths_and_short_names(data_directory, season, teams_file)

Processing season: 2020-21
Updated Fantasy-Premier-League/data/2020-21/players/MID_Gareth_Bale_1694/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ainsley_Maitland-Niles_1231/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Nathan_Redmond_780/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jacob_Ramsey_53/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jensen_Weir_1575/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Michy_Batshuayi_1354/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Ashley_Westwood_1342/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Chris_Smalling_1652/gw.csv with team strengths and short names.
Updated Fantas

Updated Fantasy-Premier-League/data/2020-21/players/DEF_Matty_Cash_32/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Meritan_Shabani_1746/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Callum_Hudson-Odoi_434/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Willy_Boly_425/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Niels_Nkounkou_1124/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Björn_Engels_1306/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Robert_Sánchez_185/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Diogo_Dalot Teixeira_369/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/pl

Updated Fantasy-Premier-League/data/2020-21/players/DEF_Kyle_Bartley_1704/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Facundo_Pellistri Rebollo_384/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Joe_Rodon_1032/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_André-Frank_Zambo Anguissa_1611/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_John_Egan_999/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Allan_Tchaptchet_1687/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Nigel_Lonwijk_1750/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Tyrick_Mitchell_210/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League

Updated Fantasy-Premier-League/data/2020-21/players/GK_Nathan_Trott_1737/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Neal_Maupay_229/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Marcus_Rashford_385/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ben_Davies_484/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_James_McCarthy_1590/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Reece_James_172/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Willy_Caballero_1244/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Ederson_Santana de Moraes_347/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/pla

Updated Fantasy-Premier-League/data/2020-21/players/MID_Tanguy_Ndombélé Alvaro_1030/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Pablo_Fornals Malla_1043/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Michail_Antonio_512/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Phil_Foden_348/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Mateusz_Klich_1162/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Carney_Chukwuemeka_161/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Wilfried_Zaha_1113/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_George_Baldock_993/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/d

...

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Riccardo_Calafiori_578/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Bilal_El Khannouss_635/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Tony_Yogane_661/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Joe_Aribo_452/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kristoffer_Ajer_86/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ben_Doak_318/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kurt_Zouma_532/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Micky_van de Ven_506/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_

Updated Fantasy-Premier-League/data/2024-25/players/MID_Anthony_Gordon_398/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Lamare_Bogarde_598/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nathan_Collins_88/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Anthony_Elanga_432/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Illia_Zabarnyi_85/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Taiwo_Awoniyi_424/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Remi_Matthews_209/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Carlos Roberto_Forbs Borges_655/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024

Updated Fantasy-Premier-League/data/2024-25/players/MID_Jaden_Philogene_569/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Mark_Flekken_91/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Son_Heung-min_503/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Joe_White_419/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Garang_Kuol_406/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_George_Edmundson_269/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Luke_Cundle_540/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Josh_Dasilva_90/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Tommy_Set

Updated Fantasy-Premier-League/data/2024-25/players/GK_Vítezslav_Jaros_660/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Leon_Bailey_28/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nathan_Aké_341/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Brennan_Johnson_491/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Dominik_Szoboszlai_336/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Joshua_Zirkzee_389/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Kai_Havertz_4/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Rodrigo_Martins Gomes_563/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/playe

Updated Fantasy-Premier-League/data/2024-25/players/MID_Morgan_Gibbs-White_433/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Enzo_Fernández_168/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Carlos_Soler_654/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Max_Kilman_524/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Marcos_Senesi_79/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kortney_Hause_40/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Taylor_Harwood-Bellis_461/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Sam_Szmodics_611/gw.csv with team strengths and short names.
Updated Fantasy-Premier-League/data/2024-25/playe

In [7]:
# We also can take into consideration whether there were fans at the stadium (during covid the 
# home team advantage wasn't so strong)
def add_crowds_to_gw(data_directory, season):
    no_crowds_start = datetime(2020, 3, 15, tzinfo=timezone.utc)
    no_crowds_end = datetime(2021, 6, 17, tzinfo=timezone.utc)

    players_folder = os.path.join(data_directory, season, "players")
    if not os.path.exists(players_folder):
        print(f"Players folder not found for season {season}. Skipping.")
        return
    
    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")
        
        if os.path.exists(gw_file_path):
            try:
                gw_data = pd.read_csv(gw_file_path)

                gw_data['kickoff_time'] = pd.to_datetime(gw_data['kickoff_time']).dt.tz_convert('UTC')

                gw_data['crowds'] = gw_data['kickoff_time'].apply(
                    lambda x: 0 if no_crowds_start <= x <= no_crowds_end else 1
                )

                gw_data.to_csv(gw_file_path, index=False)
                print(f"Updated {gw_file_path} with crowds column.")
            except Exception as e:
                print(f"Error processing {gw_file_path}: {e}")

data_directory = "Fantasy-Premier-League/data"
seasons = ["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"]

for season in seasons:
    print(f"Processing season: {season}")
    add_crowds_to_gw(data_directory, season)

Processing season: 2021-22
Updated Fantasy-Premier-League/data/2021-22/players/MID_Nathan_Redmond_780/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jacob_Ramsey_53/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Michy_Batshuayi_1354/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Ashley_Westwood_1342/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Kaide_Gordon_867/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Ryan_Bertrand_1146/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Alex_Kral_1530/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Japhet_Tanganga_1034/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Steven_Bergwijn_1255/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_

Updated Fantasy-Premier-League/data/2021-22/players/MID_Pablo_Hernández Domínguez_1381/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Jadon_Sancho_386/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Conor_Hourihane_1304/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Rúben_da Silva Neves_1269/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Mason_Holgate_225/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Juan_Mata_1415/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Donny_van de Beek_934/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Harvey_Barnes_392/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Miguel_Almirón Rejala_391/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Cen

Updated Fantasy-Premier-League/data/2021-22/players/FWD_Cameron_Archer_27/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Michael_Keane_228/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Mohammed_Salisu_1243/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Hakim_Ziyech_797/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Matthew_Cox_725/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Antonio_Rüdiger_1355/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Liam_Delap_268/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Sead_Kolasinac_1285/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Christian_Pulisic_794/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Christian_Marques_1543/gw.csv 

Updated Fantasy-Premier-League/data/2021-22/players/DEF_Phil_Bardsley_1337/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Joelinton Cássio_Apolinário de Lira_403/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Luke_Shaw_387/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Joe_Willock_420/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Wesley_Fofana_189/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Ryan_Giles_901/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Tanguy_Ndombélé Alvaro_1030/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Pablo_Fornals Malla_1043/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Michail_Antonio_512/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Ph

Updated Fantasy-Premier-League/data/2021-22/players/GK_Mark_Gillespie_948/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/FWD_Eddie_Nketiah_11/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Jonathan_Tomkinson_1452/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Pierre_Ekwah_1532/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_James_Tarkowski_236/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Pierre-Emile_Højbjerg_490/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/GK_Edouard_Mendy_1104/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Arthur_Masuaku_1263/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/DEF_Bobby_Thomas_772/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2021-22/players/MID_Paris_Maghoma_674/g

...

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Dara_O'Shea_630/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Paul_Onuachu_470/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Marcus_Tavernier_83/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Joe_Hodge_551/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Omar_Richards_438/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Mason_Holgate_225/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Harvey_Barnes_392/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Miguel_Almirón Rejala_391/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Vitalii_Mykolenko_231/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Myles_Lewis-Skelly_597/gw.cs

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Rico_Lewis_355/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Mohammed_Kudus_525/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ben_Brereton Díaz_584/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Reece_James_172/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Ederson_Santana de Moraes_347/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Lloyd_Kelly_404/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Harvey_Elliott_319/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_James_McConnell_330/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Ali_Al-Hamadi_260/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Moisés_Caicedo Corozo_1

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nathan_Collins_88/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Anthony_Elanga_432/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Illia_Zabarnyi_85/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Taiwo_Awoniyi_424/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Remi_Matthews_209/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Carlos Roberto_Forbs Borges_655/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Altay_Bayindir_367/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_João Pedro_Junqueira de Jesus_129/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Will_Hughes_203/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Isaac_H

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jayden_Meghoma_649/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Matt_Targett_416/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jan_Bednarek_455/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Armel_Bella-Kotchap_456/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Arijanet_Muric_568/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Joe_Gomez_322/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kyle_Walker_363/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Luke_Woolfenden_284/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Alfie_Gilchrist_170/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Conor_Chaplin_265/gw.csv with cr

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Odeluga_Offiah_139/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Rodrigo_Bentancur_480/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Raheem_Sterling_186/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Jason_Steele_142/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Dominic_Solanke-Mitchell_82/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Caoimhin_Kelleher_325/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_James_McAtee_357/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Wes_Burns_264/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nathan_Wood-Gordon_477/gw.csv with crowds column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Jordan_Pickford_23

In [3]:
# Let's add season number
def add_season_to_gw(data_directory, season):
    # Extract the start year from the season folder (e.g., "2021-22" -> "2021")
    season_year = season.split("-")[0]

    players_folder = os.path.join(data_directory, season, "players")
    if not os.path.exists(players_folder):
        print(f"Players folder not found for season {season}. Exiting.")
        return

    # Process each player's gw.csv
    for player_folder in os.listdir(players_folder):
        player_folder_path = os.path.join(players_folder, player_folder)
        gw_file_path = os.path.join(player_folder_path, "gw.csv")
        
        if os.path.exists(gw_file_path):
            try:
                # Load player's gw.csv
                gw_data = pd.read_csv(gw_file_path)

                # Add the season column
                gw_data['season'] = int(season_year)

                # Save the updated gw.csv
                gw_data.to_csv(gw_file_path, index=False)
                print(f"Updated {gw_file_path} with season column.")
            except Exception as e:
                print(f"Error processing {gw_file_path}: {e}")

                    
data_directory = "Fantasy-Premier-League/data"
seasons = ["2020-21", "2021-22", "2022-23", "2023-24", "2024-25"]

for season in seasons:
    print(f"Processing season: {season}")
    add_season_to_gw(data_directory, season)

Processing season: 2020-21
Updated Fantasy-Premier-League/data/2020-21/players/MID_Gareth_Bale_1694/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ainsley_Maitland-Niles_1231/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Nathan_Redmond_780/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jacob_Ramsey_53/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Jensen_Weir_1575/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Michy_Batshuayi_1354/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Ashley_Westwood_1342/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Chris_Smalling_1652/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ryan_Bertrand_1146/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/play

Updated Fantasy-Premier-League/data/2020-21/players/MID_Callum_Hudson-Odoi_434/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Willy_Boly_425/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Niels_Nkounkou_1124/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Björn_Engels_1306/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Robert_Sánchez_185/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Diogo_Dalot Teixeira_369/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Martin_Kelly_1361/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Marcus_Bettinelli_155/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Teddy_Jenks_1573/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Filip_Benkovic_1621/gw

Updated Fantasy-Premier-League/data/2020-21/players/DEF_John_Egan_999/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Allan_Tchaptchet_1687/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Nigel_Lonwijk_1750/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Tyrick_Mitchell_210/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Stuart_Armstrong_1234/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Vontae_Daley-Campbell_1380/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Dan_Burn_395/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Paulo_Gazzaniga Farias_1136/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Thomas_Partey_20/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Grady_Diangana_1

Updated Fantasy-Premier-League/data/2020-21/players/MID_Harvey_Elliott_319/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Anthony_Knockaert_843/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ezgjan_Alioski_1630/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Simon_Moore_1670/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Mahmoud Ahmed_Ibrahim Hassan_1307/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Douglas Luiz_Soares de Paulo_692/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Ahmed_El Mohamady_1556/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Moisés_Caicedo Corozo_157/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Joshua_King_854/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/

Updated Fantasy-Premier-League/data/2020-21/players/DEF_Kean_Bryan_1677/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/GK_Martin_Dúbravka_396/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Ayoze_Pérez_1150/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Andrew_Robertson_335/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_Federico_Fernández_1214/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Rodrigo 'Rodri'_Hernandez_360/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/DEF_James_Justin_295/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Solly_March_132/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/FWD_Florin_Andone_1329/gw.csv with season column.
Updated Fantasy-Premier-League/data/2020-21/players/MID_Conor_Gallagher_16

...

Updated Fantasy-Premier-League/data/2024-25/players/MID_Mikkel_Damsgaard_89/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Yerson_Mosquera_558/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Jamie_Vardy_306/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Kaden_Rodney_647/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Roman_Dixon_627/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Ola_Aina_422/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Christian_Eriksen_370/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Steven_Benda_242/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jack_Clarke_631/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Mads_Roerslev Rasmussen_105/gw.csv with s

Updated Fantasy-Premier-League/data/2024-25/players/MID_Abdul_Fatawu_570/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Imari_Samuels_663/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Flynn_Downes_518/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Cole_Palmer_182/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Jamal_Lewis_408/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Myles_Peart-Harris_103/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kosta_Nedeljković_50/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Dwight_McNeil_230/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jack_Grealish_349/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Boubacar_Kamara_42/gw.csv with se

Updated Fantasy-Premier-League/data/2024-25/players/FWD_Iliman_Ndiaye_232/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_William_Saliba_18/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Idrissa_Gueye_222/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Harry_Souttar_303/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Felipe_Rodrigues da Silva_653/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Noni_Madueke_177/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Elliot_Anderson_423/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Ismaïla_Sarr_585/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kyle_Walker-Peters_476/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Jaden_Philogene_56

Updated Fantasy-Premier-League/data/2024-25/players/MID_Eric_da Silva Moreira_428/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Ryan_Manning_467/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_João Victor_Gomes da Silva_553/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Joe_Gauci_39/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Rob_Holding_202/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Emil_Krafth_405/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Vítezslav_Jaros_660/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Leon_Bailey_28/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Nathan_Aké_341/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Brennan_Johnson_491/gw.csv wit

Updated Fantasy-Premier-League/data/2024-25/players/DEF_Marcos_Senesi_79/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Kortney_Hause_40/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Taylor_Harwood-Bellis_461/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Sam_Szmodics_611/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Lino_da Cruz Sousa_56/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Alisson_Ramses Becker_310/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/DEF_Pervis_Estupiñán_122/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/MID_Sean_Longstaff_410/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/GK_Bernd_Leno_248/gw.csv with season column.
Updated Fantasy-Premier-League/data/2024-25/players/FWD_Adam_Armstrong_453/